# Домашнє завдання: Прогнозування орендної плати за житло

## Мета завдання
Застосувати знання з лекції для побудови моделі лінійної регресії, що прогнозує орендну плату за житло в Індії. Ви пройдете весь цикл вирішення задачі машинного навчання: від дослідницького аналізу до оцінки якості моделі.

## Опис датасету
**House Rent Prediction Dataset** містить інформацію про 4700+ оголошень про оренду житла в Індії з такими параметрами:
- **BHK**: Кількість спалень, залів, кухонь
- **Rent**: Орендна плата (цільова змінна)
- **Size**: Площа в квадратних футах
- **Floor**: Поверх та загальна кількість поверхів
- **Area Type**: Тип розрахунку площі
- **Area Locality**: Район
- **City**: Місто
- **Furnishing Status**: Стан меблювання
- **Tenant Preferred**: Тип орендаря
- **Bathroom**: Кількість ванних кімнат
- **Point of Contact**: Контактна особа

---

## Завдання 1: Завантаження та перший огляд даних (1 бал)

**Що потрібно зробити:**
1. Завантажте дані з файлу `House_Rent_Dataset.csv`
2. Виведіть розмір датасету
3. Покажіть перші 5 рядків
4. Виведіть загальну інформацію про дані (включно з типами даних та кількістю значень)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = 'drive/MyDrive/Data_analitic/Python/House_Rent_Dataset.csv'
House_Rent_Dataset = pd.read_csv(data_path)

In [5]:
House_Rent_Dataset.shape

(4746, 12)

In [6]:
House_Rent_Dataset.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


In [7]:
House_Rent_Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Posted On          4746 non-null   object
 1   BHK                4746 non-null   int64 
 2   Rent               4746 non-null   int64 
 3   Size               4746 non-null   int64 
 4   Floor              4746 non-null   object
 5   Area Type          4746 non-null   object
 6   Area Locality      4746 non-null   object
 7   City               4746 non-null   object
 8   Furnishing Status  4746 non-null   object
 9   Tenant Preferred   4746 non-null   object
 10  Bathroom           4746 non-null   int64 
 11  Point of Contact   4746 non-null   object
dtypes: int64(4), object(8)
memory usage: 445.1+ KB


## Завдання 2: Дослідницький аналіз даних (EDA) (4 бали)

**Що потрібно зробити:**
1. **Аналіз пропущених значень.** Перевірте наявність і відсоток пропущених значень у кожній колонці
2. **Базова статистика.** Обчисліть базову статистику (середнє, квартилі, стандартне відхилення) для числових змінних.
3. **Аналіз цільової змінної.** Побудуйте гістограму розподілу цільової змінної (Rent)
4. **Робота з викидами.** Знайдіть та видаліть викиди в цільовій змінній (якщо є). Визначити викиди можна будь-яким зрозумілим для вас способом, як варіант - таким, що використовується в побудові box-plot (https://en.wikipedia.org/wiki/Box_plot#Example_with_outliers).
5. **Аналіз категоріальних змінних.** Виведіть кількість унікальних значень для кожної з категоріальних колонок.


In [8]:
missing_values = House_Rent_Dataset.isnull().sum()
missing_values_persent = (missing_values/len(House_Rent_Dataset)) * 100
missing_values_persent

,0
Posted On,0.0
BHK,0.0
Rent,0.0
Size,0.0
Floor,0.0
Area Type,0.0
Area Locality,0.0
City,0.0
Furnishing Status,0.0
Tenant Preferred,0.0


In [9]:
stat = House_Rent_Dataset[['BHK', 'Rent', 'Size', 'Bathroom']].describe()
stat.round(3)

,BHK,Rent,Size,Bathroom
count,4746.000,4746.000,4746.000,4746.000
mean,2.084,34993.451,967.491,1.966
std,0.832,78106.413,634.202,0.885
min,1.000,1200.000,10.000,1.000
25%,2.000,10000.000,550.000,1.000
50%,2.000,16000.000,850.000,2.000
75%,3.000,33000.000,1200.000,2.000
max,6.000,3500000.000,8000.000,10.000


In [10]:
fig = px.histogram(
    House_Rent_Dataset,
    x='Rent',
    nbins=100,
    title='Розподіл цільової змінної (Rent)',
    labels={'Rent': 'Орендна плата', 'count': 'Кількість'}
)

fig.update_layout(
    showlegend=False,
    height=400
)

fig.show()


In [11]:
Q1 = House_Rent_Dataset['Rent'].quantile(0.25)
Q3 = House_Rent_Dataset['Rent'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

House_Rent_Dataset_cleaned = House_Rent_Dataset[(House_Rent_Dataset['Rent'] >= lower_bound) &
                                                (House_Rent_Dataset['Rent'] <= upper_bound)]

print(f'Розмір датасету після видалення викидів: {House_Rent_Dataset_cleaned.shape}')


Розмір датасету після видалення викидів: (4226, 12)


In [12]:
House_Rent_Dataset.select_dtypes('object').nunique()

,0
Posted On,81
Floor,480
Area Type,3
Area Locality,2235
City,6
Furnishing Status,3
Tenant Preferred,3
Point of Contact,3



## Завдання 3: Аналіз кореляцій та взаємозв'язків (3 бали)

**Що потрібно зробити:**
1. Обчисліть матрицю кореляцій для числових змінних
2. Візуалізуйте кореляційну матрицю за допомогою heatmap
3. Побудуйте scatter plot між Size та Rent
4. Проаналізуйте взаємозв'язок між BHK та Rent за допомогою boxplot (який розподіл плати для різних значень BHK)


In [13]:
correlation_matrix = House_Rent_Dataset[['BHK', 'Rent', 'Size', 'Bathroom']].corr()

In [14]:
fig = px.imshow(
    correlation_matrix,
    text_auto='.2f',
    color_continuous_scale='RdBu_r',
    title='Кореляція між метриками взаємодії',
    labels=dict(color="Кореляція")
)
fig.update_layout(height=500)
fig.show()

In [15]:
fig = px.scatter(House_Rent_Dataset, x='Size', y='Rent', title='Scatter plot: Size vs Rent')
fig.show()

In [16]:
fig = px.box(House_Rent_Dataset, x='BHK', y='Rent', title='Розподіл Rent для різних значень BHK')
fig.show()

- зі зростанням BHK зростає і орендна плата.
- 1-2 BHK — найбільш доступні варіанти, популярні серед орендарів з обмеженим бюджетом.
- 3 BHK має найвищу верхню межу (викиди до 3.5 млн) — можливо, через наявність елітних квартир.
- 5 BHK демонструє найстабільнішу структуру з великою кількістю даних.
- в усіх групах є викиди (аномально висока орендна плата) окрім категорії 5 BHK
- навіть у межах однкової кількості кімнат рента може сильно варіюватися, що свідчить про вплив додаткових факторів — таких як локація, площа, стан житла тощо.

## Завдання 4: Feature Engineering та підготовка даних (4 бали)

**Що потрібно зробити:**
1. Закодуйте категоріальні змінні за допомогою One-Hot Encoding. Пригадайте, що в лекції ми говорили щодо кодування кат. змінних з великої кількістю різних значень і як працювати з такими випадками. Ви можете закодувати не всі кат. змінні, а лише ті, що вважаєте за потрібні (скажімо ті, що мають відносно небагато різних значень).
2. **Опціонально (по 0.5 бала за кожну доцільну ознаку):** Додайте нові ознаки, обчислені на основі наявних даних, які б на ваш погляд були корисними для моделі
3. Виберіть ознаки для побудови моделі (виключіть непотрібні колонки). Виключити можна, наприклад, ті колонки, які мають категоріальний тип і забагато (більше 20) різних значень. Треба виключити хоча б 1 колонку.
4. Розділіть дані на ознаки (X) та цільову змінну (y)
5. Застосуйте стандартизацію до числових ознак


In [17]:
House_Rent_Dataset['City'].value_counts()

,count
City,
Mumbai,972
Chennai,891
Bangalore,886
Hyderabad,868
Delhi,605
Kolkata,524


In [18]:
House_Rent_Dataset['Area Locality'].value_counts()

,count
Area Locality,
Bandra West,37
Gachibowli,29
Electronic City,24
"Miyapur, NH 9",22
Velachery,22
...,...
Hoysala Nagar,1
Nagarabhavi,1
"Shamanna Garden, Wilson Garden",1


In [19]:
House_Rent_Dataset['Area Type'].value_counts()

,count
Area Type,
Super Area,2446
Carpet Area,2298
Built Area,2


In [20]:
House_Rent_Dataset['Furnishing Status'].value_counts()

,count
Furnishing Status,
Semi-Furnished,2251
Unfurnished,1815
Furnished,680


####Пояснення:
я вирішила надати категорії Furnishing Status ранг, оскільки вважаю, що наявність/відсутність меблювання впливає на орендну плату


In [21]:
furnishing_map = {
    'Unfurnished': 0,
    'Semi-Furnished': 1,
    'Furnished': 2}

House_Rent_Dataset['Furnishing_Status_Ordinal'] = House_Rent_Dataset['Furnishing Status'].map(furnishing_map)

In [22]:
def floor_to_num(floor_str):
    if floor_str == 'Ground out of 2':
        return 0
    elif floor_str == 'Upper Basement':
        return -1
    elif floor_str == 'Lower Basement':
        return -2
    else:
        try:
            floor_num_str = floor_str.split(' out of ')[0]
            # Якщо це "Ground"
            if floor_num_str == 'Ground':
                return 0
            elif floor_num_str == 'Upper Basement':
                return -1
            elif floor_num_str == 'Lower Basement':
                return -2
            else:
                return int(floor_num_str)
        except:
            return None

def total_floors_to_num(floor_str):
    try:
        total_floors_str = floor_str.split(' out of ')[1]
        return int(total_floors_str)
    except:
        return None

# Застосування
House_Rent_Dataset['Floor Level'] = House_Rent_Dataset['Floor'].apply(floor_to_num)
House_Rent_Dataset['Total Floors'] = House_Rent_Dataset['Floor'].apply(total_floors_to_num)

In [23]:
categorical_cols = ['City', 'Tenant Preferred']

dummies_columns = pd.get_dummies(House_Rent_Dataset[categorical_cols], drop_first=False).astype(int)

In [24]:
df_HRD_new = pd.concat([House_Rent_Dataset.drop(columns=categorical_cols), dummies_columns], axis=1)

In [25]:
df_HRD_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Posted On                          4746 non-null   object 
 1   BHK                                4746 non-null   int64  
 2   Rent                               4746 non-null   int64  
 3   Size                               4746 non-null   int64  
 4   Floor                              4746 non-null   object 
 5   Area Type                          4746 non-null   object 
 6   Area Locality                      4746 non-null   object 
 7   Furnishing Status                  4746 non-null   object 
 8   Bathroom                           4746 non-null   int64  
 9   Point of Contact                   4746 non-null   object 
 10  Furnishing_Status_Ordinal          4746 non-null   int64  
 11  Floor Level                        4746 non-null   int64

In [26]:
df_HRD_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4746 entries, 0 to 4745
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Posted On                          4746 non-null   object 
 1   BHK                                4746 non-null   int64  
 2   Rent                               4746 non-null   int64  
 3   Size                               4746 non-null   int64  
 4   Floor                              4746 non-null   object 
 5   Area Type                          4746 non-null   object 
 6   Area Locality                      4746 non-null   object 
 7   Furnishing Status                  4746 non-null   object 
 8   Bathroom                           4746 non-null   int64  
 9   Point of Contact                   4746 non-null   object 
 10  Furnishing_Status_Ordinal          4746 non-null   int64  
 11  Floor Level                        4746 non-null   int64

In [27]:
df_HRD_new['Total Floors'] = df_HRD_new['Total Floors'].fillna(df_HRD_new['Total Floors'].median())

###Пояснення:
- видалення колонок зробила тільки на цьому етапі, бо робила завдання послідовно.

In [28]:
df_model = df_HRD_new.drop(columns=['Area Locality'])

In [29]:
df_model['Posted On'] = pd.to_datetime(df_model['Posted On'], errors='coerce')

# Витягуємо рік і місяць
df_model['Posted_Year'] = df_model['Posted On'].dt.year
df_model['Posted_Month'] = df_model['Posted On'].dt.month

In [30]:
df_model.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Furnishing Status', 'Bathroom', 'Point of Contact',
       'Furnishing_Status_Ordinal', 'Floor Level', 'Total Floors',
       'City_Bangalore', 'City_Chennai', 'City_Delhi', 'City_Hyderabad',
       'City_Kolkata', 'City_Mumbai', 'Tenant Preferred_Bachelors',
       'Tenant Preferred_Bachelors/Family', 'Tenant Preferred_Family',
       'Posted_Year', 'Posted_Month'],
      dtype='object')

In [31]:
# Обчислюємо кореляції з цільовою змінною
corr_matrix = df_model.corr(numeric_only=True)
top_corr = corr_matrix['Rent'].abs().sort_values(ascending=False)

# Відкидаємо саму Rent
top_corr = top_corr.drop('Rent')

# Вибираємо ознаки з кореляцією > 0.01
top_corr_features = top_corr[top_corr > 0.01].index.tolist()

In [32]:
top_corr_features

['Bathroom',
 'Size',
 'BHK',
 'Total Floors',
 'City_Mumbai',
 'Floor Level',
 'Furnishing_Status_Ordinal',
 'City_Kolkata',
 'Posted_Month',
 'City_Hyderabad',
 'City_Chennai',
 'Tenant Preferred_Bachelors/Family',
 'Tenant Preferred_Family',
 'City_Bangalore',
 'Tenant Preferred_Bachelors',
 'City_Delhi']

In [33]:
len(top_corr_features)

16

In [34]:
#city_dummies = df_model.filter(like='City_').columns.tolist()
#tenant_dummies = df_model.filter(like='Tenant Preferred_').columns.tolist()

In [35]:
features = top_corr_features
# Визначаємо X (ознаки) та y (ціль)
X = df_model[features]  # Ознаки
y = df_model['Rent']  # Цільова змінна

print(f"\nРозмір X (ознак): {X.shape}")
print(f"Розмір y (цілі): {y.shape}")


Розмір X (ознак): (4746, 16)
Розмір y (цілі): (4746,)


In [36]:
# Створюємо та застосовуємо скейлер
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Перетворюємо назад у DataFrame для зручності
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

## Завдання 5: Розділення даних та навчання моделі (3 бали)

**Що потрібно зробити:**
1. Розділіть дані на навчальну (80%) та тестову (20%) вибірки.
2. Створіть модель лінійної регресії.
3. Навчіть модель на навчальних даних.
4. Виведіть усі коефіцієнти моделі (ваги) та напишіть, які 2 ознаки найбільше впливають на прогноз.
5. Зробіть прогнози на тренувальній та тестовій вибірках.

In [37]:
# Розділяємо дані: 80% на навчання, 20% на тест
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled_df, y,
    test_size=0.2,  # 20% даних йде на тест
    random_state=42  # фіксуємо випадковість для відтворюваності
)

In [38]:
print('Пропущені значення у X_train:')
print(X_train.isna().sum()[X_train.isna().sum() > 0])
#робила перевірку, бо були пропущені дані

Пропущені значення у X_train:
Series([], dtype: int64)


In [39]:
from sklearn.linear_model import LinearRegression

# Створюємо модель
model = LinearRegression()

# Навчаємо модель на навчальних даних
model.fit(X_train, y_train)

LinearRegression()

In [40]:
# Виводимо ваги для кожної ознаки
for feature, weight in zip(model.feature_names_in_, model.coef_):
    print(f"{feature}: {weight:.2f}")

print(f"\nЗміщення (intercept): {model.intercept_:.2f}")

Bathroom: 9107.07
Size: 23496.48
BHK: 2706.43
Total Floors: 3050.29
City_Mumbai: 16581.95
Floor Level: 5944.72
Furnishing_Status_Ordinal: 2231.01
City_Kolkata: -2050.79
Posted_Month: -9.98
City_Hyderabad: -9027.60
City_Chennai: -5278.81
Tenant Preferred_Bachelors/Family: 360.84
Tenant Preferred_Family: -2807.27
City_Bangalore: -2509.81
Tenant Preferred_Bachelors: 1787.80
City_Delhi: 1439.68

Зміщення (intercept): 35206.91


In [41]:
coef_pairs = list(zip(model.feature_names_in_, model.coef_))

coef_pairs_sorted_abs = sorted(coef_pairs, key=lambda x: abs(x[1]), reverse=True)

print("\nТОП-5 найвпливовіших ознак (за абсолютним значенням):")



ТОП-5 найвпливовіших ознак (за абсолютним значенням):


In [42]:
# Відфільтруємо лише позитивні коефіцієнти
positive_coefs = [(f, w) for f, w in coef_pairs if w > 0]

# Відсортуємо за спаданням (від найбільшого до меншого)
positive_coefs_sorted = sorted(positive_coefs, key=lambda x: x[1], reverse=True)

print("ТОП-5 ознак з найбільшим позитивним впливом:")
for feature, weight in positive_coefs_sorted[:5]:
    print(f"{feature}: {weight:.2f}")


ТОП-5 ознак з найбільшим позитивним впливом:
Size: 23496.48
City_Mumbai: 16581.95
Bathroom: 9107.07
Floor Level: 5944.72
Total Floors: 3050.29


### Найбільші 2 ознаки, що впливають значення ренти - Size: 23496.48 та City_Mumbai: 16581.95
- але я би ще розглядала Bathroom: 9107.07, як один з головних компонентів впливу на ренту.

In [43]:
# Прогнози на навчальній вибірці
y_train_pred = model.predict(X_train)

# Прогнози на тестовій вибірці (нові дані!)
y_test_pred = model.predict(X_test)

# Порівняння перших 10 прогнозів з реальністю
comparison = pd.DataFrame({
    'Реальна рента': y_test.values[:20],
    'Прогнозована рента': y_test_pred[:20].round(0),
    'Помилка': (y_test.values[:20] - y_test_pred[:20]).round(0)
})
print("Приклади прогнозів на тестовій вибірці:")
print(comparison)

Приклади прогнозів на тестовій вибірці:
    Реальна рента  Прогнозована рента   Помилка
0           16000             28935.0  -12935.0
1           12000             16951.0   -4951.0
2           28000             63840.0  -35840.0
3            8000             66067.0  -58067.0
4           46000             77895.0  -31895.0
5           17000             14138.0    2862.0
6           57000             81002.0  -24002.0
7            9500             18030.0   -8530.0
8          400000            167624.0  232376.0
9           15000              8134.0    6866.0
10          10000             -2175.0   12175.0
11          12000             -5363.0   17363.0
12          27000             34434.0   -7434.0
13          16000             -1411.0   17411.0
14          15000             29369.0  -14369.0
15           8500             47161.0  -38661.0
16         140000            107139.0   32861.0
17          12000             -9377.0   21377.0
18           6000              4378.0    1622.0


## Завдання 6: Оцінка якості моделі (2 бали)

**Що потрібно зробити:**
1. Обчисліть MAE, RMSE та R² для навчальної та тестової вибірок
2. Порівняйте метрики та зробіть висновок про якість моделі
3. Проаналізуйте і дайте висновок, чи є ознаки перенавчання або недонавчання (**Нагадування**: перенавчання - коли модель дуже добре працює на тренувальних даних, але погано на тестових; недонавчання - коли модель погано працює навіть на тренувальних даних)
4. Побудуйте графік розсіювання "реальні vs прогнозовані значення" та зробіть висновок про якість моделі


In [44]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Розраховуємо метрики для тестової вибірки
mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_test_pred)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):')
print('='*50)
print(f'\nMAE: {mae:.2f} рента')
print(f'RMSE: {rmse:.2f} рента')
print(f'R²: {r2:.3f}')

# Порівняння з навчальною вибіркою
mae = mean_absolute_error(y_train, y_train_pred)
mse = mean_squared_error(y_train, y_train_pred)
rmse = np.sqrt(mse)
r2_train = r2_score(y_train, y_train_pred)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:')
print('='*50)
print(f'\nMAE: {mae:.2f} рента')
print(f'RMSE: {rmse:.2f} рента')
print(f'R²: {r2_train:.3f}')

МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):

MAE: 21930.11 рента
RMSE: 43783.20 рента
R²: 0.519
МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:

MAE: 22430.30 рента
RMSE: 68620.24 рента
R²: 0.290


In [45]:
np.mean(y_train).round(2)

np.float64(35151.52)

###Висновок:
- модель не демонструє переваги на тренуванні R²: 0.290 (тобто не переадаптувалась), і водночас не показує провалу на тесті R²: 0.519.
- є дивна ситуація: R² на тесті кращий, ніж на тренуванні (можливо через наявність викидів).

In [46]:
# Візуалізація: реальні vs прогнозовані значення
fig = px.scatter(
    x=y_test,
    y=y_test_pred,
    title='Реальні vs Прогнозовані значення ренти (тестова вибірка)',
    labels={'x': 'Реальні значення ренти', 'y': 'Прогнозовані значення ренти'},
    opacity=0.6
)

# Додаємо ідеальну лінію (де прогноз = реальність)
max_val = max(y_test.max(), y_test_pred.max())
fig.add_trace(
    go.Scatter(
        x=[0, max_val],
        y=[0, max_val],
        mode='lines',
        name='Ідеальний прогноз',
        line=dict(color='red', dash='dash')
    )
)

fig.update_layout(height=500)
fig.show()

###Висновок
- модель систематично недооцінює значення ренти, особливо в діапазоні від 200k і вище.
- значення ренти в низькому діапазоні (до 100К) - переоцінює.
- можливо є вплив аномальних значень ренти і їх треба оцінювати окремо і виключити із загальної вибірки.

## Завдання 7: Аналіз помилок (4 бали)

**Що потрібно зробити:**
1. Обчисліть помилки (residuals = реальні - прогнозовані значення)
2. Побудуйте гістограму розподілу помилок
3. Створіть scatter plot помилок відносно величини прогнозованих значень. Чи росте помилка з ростом прогнозованого значення?
4. Знайдіть 5 прогнозів з найбільшими помилками
5. Проаналізуйте, на яких типах житла модель помиляється найбільше. Типи можна розрізняти за кількістю кімнат чи містом, наприклад.
6. Подумайте і напишіть, які наступні кроки ви б зробили, аби поліпшити якість моделі. Опціонально можна їх зробити і ми перевіримо :)

In [47]:
# Розраховуємо помилки (залишки)
residuals = y_test - y_test_pred

# Гістограма помилок
fig = px.histogram(
    x=residuals,
    nbins=50,
    title='Розподіл помилок прогнозування',
    labels={'x': 'Помилка (реальні - прогнозовані)', 'count': 'Кількість'},
    color_discrete_sequence=['#e74c3c']
)
fig.add_vline(x=0, line_dash='dash', line_color='black', annotation_text='Ідеальний прогноз')
fig.update_layout(height=400)
fig.show()

In [48]:
# Scatter plot: помилки vs прогнозовані значення
fig = px.scatter(
    x=y_test_pred,
    y=residuals,
    title='Залежність помилок від прогнозованих значень',
    labels={'x': 'Прогнозовані значення ренти', 'y': 'Помилка'},
    opacity=0.5
)

# Додаємо горизонтальну лінію на 0
fig.add_hline(y=0, line_dash='dash', line_color='red', annotation_text='Без помилки')

fig.update_layout(height=400)
fig.show()

###Чи росте помилка з ростом прогнозованого значення?
- похибка має тенденцію зростати при зростанні прогнозованого значення ренти, але не лінійно — розкиданість точок(дисперсія) стосовно червоної лінії  зростає. Це означає, що модель менш стабільна на дорогих об'єктах: вона може або сильно недооцінити, або іноді переоцінити ціни.

In [49]:
# Знаходимо пости з найбільшими помилками
errors_df = pd.DataFrame({
    'real': y_test.values,
    'predicted': y_test_pred,
    'error': np.abs(residuals)
})

# Топ-5 найбільших помилок
top_errors = errors_df.nlargest(5, 'error').round(2)
print('Квартири з найбільшимим помилками прогнозування ренти:')
print(top_errors)

Квартири з найбільшимим помилками прогнозування ренти:
         real  predicted      error
1001  1200000  256933.71  943066.29
1344   400000  167624.08  232375.92
1718   380000  162414.47  217585.53
3148   330000  132328.59  197671.41
1084   300000  135124.58  164875.42


In [50]:
idx_list = [1001, 1344, 1718, 3148, 1084]

filtered_df = df_HRD_new.loc[idx_list]
print(filtered_df)


       Posted On  BHK     Rent  Size        Floor    Area Type  \
1001  2022-06-01    4  1200000  5000  4 out of 15  Carpet Area   
1344  2022-06-22    4   400000  2500  5 out of 12  Carpet Area   
1718  2022-06-29    4   380000  3500   2 out of 4  Carpet Area   
3148  2022-06-14    3   330000  3600   3 out of 4  Carpet Area   
1084  2022-06-10    3   300000  1800  8 out of 12  Carpet Area   

              Area Locality Furnishing Status  Bathroom Point of Contact  ...  \
1001                   Juhu    Semi-Furnished         4    Contact Agent  ...   
1344            Bandra West         Furnished         4    Contact Agent  ...   
1718           Lavelle Road    Semi-Furnished         5    Contact Agent  ...   
3148  Madras Boat Club Road    Semi-Furnished         3    Contact Agent  ...   
1084            Bandra West         Furnished         3    Contact Agent  ...   

      Total Floors  City_Bangalore  City_Chennai  City_Delhi  City_Hyderabad  \
1001          15.0               0  

###
- модель має найбільші похибки у прогнозуванні для квартир з великою площею (1800–5000 кв.ф.)
- високою рентою (300K+)
- розташованих переважно в Мумбаї
- найчастіше це 3–4 кімнатні, (частково) мебльовані апартаменти (можлива причина - наявність великої кількості викидів-аномальнихзначень ренти квартир)
- квартири з гнучкішим підходом до типу орендаря (Tenant Preferred_Bachelors/Family) часто мають вищі похибки у прогнозі

In [51]:
df = pd.get_dummies(df_HRD_new, columns=['Furnishing Status'], drop_first=True)



###Вирішила зробити one-hot для Furnishing Status, і перевірити точність моделі. Бо до цього давала ранг в цій категорії.

In [52]:
df_HRD_new = pd.get_dummies(df_HRD_new, columns=['Furnishing Status'], drop_first=True)

In [53]:
df_HRD_new.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'Bathroom', 'Point of Contact',
       'Furnishing_Status_Ordinal', 'Floor Level', 'Total Floors',
       'City_Bangalore', 'City_Chennai', 'City_Delhi', 'City_Hyderabad',
       'City_Kolkata', 'City_Mumbai', 'Tenant Preferred_Bachelors',
       'Tenant Preferred_Bachelors/Family', 'Tenant Preferred_Family',
       'Furnishing Status_Semi-Furnished', 'Furnishing Status_Unfurnished'],
      dtype='object')

In [54]:
df_model_1 = df_HRD_new.drop(columns=['Area Locality'])

In [55]:
df_model_1['Posted On'] = pd.to_datetime(df_model['Posted On'], errors='coerce')

# Витягуємо рік і місяць
df_model_1['Posted_Year'] = df_model_1['Posted On'].dt.year
df_model_1['Posted_Month'] = df_model_1['Posted On'].dt.month

In [56]:
df_model_1.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type', 'Bathroom',
       'Point of Contact', 'Furnishing_Status_Ordinal', 'Floor Level',
       'Total Floors', 'City_Bangalore', 'City_Chennai', 'City_Delhi',
       'City_Hyderabad', 'City_Kolkata', 'City_Mumbai',
       'Tenant Preferred_Bachelors', 'Tenant Preferred_Bachelors/Family',
       'Tenant Preferred_Family', 'Furnishing Status_Semi-Furnished',
       'Furnishing Status_Unfurnished', 'Posted_Year', 'Posted_Month'],
      dtype='object')

In [57]:
# Обчислюємо кореляції з цільовою змінною
corr_matrix_1 = df_model_1.corr(numeric_only=True)
top_corr_1 = corr_matrix['Rent'].abs().sort_values(ascending=False)

# Відкидаємо саму Rent
top_corr_1 = top_corr_1.drop('Rent')

# Вибираємо ознаки з кореляцією > 0.01
top_corr_features_1 = top_corr_1[top_corr_1 > 0.01].index.tolist()

In [58]:
len(top_corr_features_1)

16

In [59]:
features_1 = top_corr_features_1
# Визначаємо X (ознаки) та y (ціль)
X = df_model_1[features_1]  # Ознаки
y = df_model_1['Rent']  # Цільова змінна

print(f"\nРозмір X (ознак): {X.shape}")
print(f"Розмір y (цілі): {y.shape}")


Розмір X (ознак): (4746, 16)
Розмір y (цілі): (4746,)


In [60]:
# Створюємо та застосовуємо скейлер
scaler_1 = StandardScaler()
X_scaled_1 = scaler.fit_transform(X)

# Перетворюємо назад у DataFrame для зручності
X_scaled_df_1 = pd.DataFrame(X_scaled_1, columns=X.columns, index=X.index)

In [61]:
# Розділяємо дані: 80% на навчання, 20% на тест
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    X_scaled_df_1, y,
    test_size=0.2,  # 20% даних йде на тест
    random_state=42  # фіксуємо випадковість для відтворюваності
)

In [62]:
from sklearn.linear_model import LinearRegression

# Створюємо модель
model_1 = LinearRegression()

# Навчаємо модель на навчальних даних
model_1.fit(X_train_1, y_train_1)

LinearRegression()

In [63]:
# Прогнози на навчальній вибірці
y_train_pred_1 = model.predict(X_train_1)

# Прогнози на тестовій вибірці (нові дані!)
y_test_pred_1 = model.predict(X_test_1)

# Порівняння перших 10 прогнозів з реальністю
comparison_1 = pd.DataFrame({
    'Реальна рента': y_test_1.values[:20],
    'Прогнозована рента': y_test_pred_1[:20].round(0),
    'Помилка': (y_test_1.values[:20] - y_test_pred_1[:20]).round(0)
})
print("Приклади прогнозів на тестовій вибірці:")
print(comparison_1)

Приклади прогнозів на тестовій вибірці:
    Реальна рента  Прогнозована рента   Помилка
0           16000             28935.0  -12935.0
1           12000             16951.0   -4951.0
2           28000             63840.0  -35840.0
3            8000             66067.0  -58067.0
4           46000             77895.0  -31895.0
5           17000             14138.0    2862.0
6           57000             81002.0  -24002.0
7            9500             18030.0   -8530.0
8          400000            167624.0  232376.0
9           15000              8134.0    6866.0
10          10000             -2175.0   12175.0
11          12000             -5363.0   17363.0
12          27000             34434.0   -7434.0
13          16000             -1411.0   17411.0
14          15000             29369.0  -14369.0
15           8500             47161.0  -38661.0
16         140000            107139.0   32861.0
17          12000             -9377.0   21377.0
18           6000              4378.0    1622.0


In [64]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Розраховуємо метрики для тестової вибірки
mae_1 = mean_absolute_error(y_test_1, y_test_pred_1)
mse_1 = mean_squared_error(y_test_1, y_test_pred_1)
rmse_1 = np.sqrt(mse_1)
r2_1 = r2_score(y_test_1, y_test_pred_1)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):')
print('='*50)
print(f'\nMAE: {mae_1:.2f} рента')
print(f'RMSE: {rmse_1:.2f} рента')
print(f'R²: {r2_1:.3f}')

# Порівняння з навчальною вибіркою
mae_1 = mean_absolute_error(y_train_1, y_train_pred_1)
mse_1 = mean_squared_error(y_train_1, y_train_pred_1)
rmse_1 = np.sqrt(mse_1)
r2_train_1 = r2_score(y_train_1, y_train_pred_1)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:')
print('='*50)
print(f'\nMAE: {mae_1:.2f} рента')
print(f'RMSE: {rmse_1:.2f} рента')
print(f'R²: {r2_train_1:.3f}')

МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):

MAE: 21930.11 рента
RMSE: 68620.24 рента
R²: 0.519
МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:

MAE: 22430.30 рента
RMSE: 68620.24 рента
R²: 0.290


###не дивлячись на мої припущення, що ранжування могло вплинути на модель, one-hot кодування не вплинуло на якість моделі.

In [70]:
Q1 = df['Rent'].quantile(0.25)
Q3 = df['Rent'].quantile(0.75)
IQR = Q3 - Q1

# Межі для "стандартних" значень (без викидів)
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Фільтрація
df_standard = df[(df['Rent'] >= lower_bound) & (df['Rent'] <= upper_bound)].copy()
df_outliers = df[(df['Rent'] < lower_bound) | (df['Rent'] > upper_bound)].copy()

print(f'Розмір стандартної вибірки: {len(df_standard)}')
print(f'Розмір вибірки з викидами: {len(df_outliers)}')


Розмір стандартної вибірки: 4226
Розмір вибірки з викидами: 520


In [71]:
df_standard.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4226 entries, 0 to 4745
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Posted On                          4226 non-null   object 
 1   BHK                                4226 non-null   int64  
 2   Rent                               4226 non-null   int64  
 3   Size                               4226 non-null   int64  
 4   Floor                              4226 non-null   object 
 5   Area Type                          4226 non-null   object 
 6   Area Locality                      4226 non-null   object 
 7   Bathroom                           4226 non-null   int64  
 8   Point of Contact                   4226 non-null   object 
 9   Furnishing_Status_Ordinal          4226 non-null   int64  
 10  Floor Level                        4226 non-null   int64  
 11  Total Floors                       4226 non-null   float64
 1

In [72]:
df_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 520 entries, 104 to 4716
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Posted On                          520 non-null    object 
 1   BHK                                520 non-null    int64  
 2   Rent                               520 non-null    int64  
 3   Size                               520 non-null    int64  
 4   Floor                              520 non-null    object 
 5   Area Type                          520 non-null    object 
 6   Area Locality                      520 non-null    object 
 7   Bathroom                           520 non-null    int64  
 8   Point of Contact                   520 non-null    object 
 9   Furnishing_Status_Ordinal          520 non-null    int64  
 10  Floor Level                        520 non-null    int64  
 11  Total Floors                       520 non-null    float64
 

In [74]:
# Перетворення 'Posted On' у datetime
df_standard['Posted On'] = pd.to_datetime(df_standard['Posted On'], errors='coerce')

# Тепер можна витягувати рік і місяць
df_standard['Posted_Year'] = df_standard['Posted On'].dt.year
df_standard['Posted_Month'] = df_standard['Posted On'].dt.month

In [75]:
# Перетворення 'Posted On' у datetime
df_outliers['Posted On'] = pd.to_datetime(df_outliers['Posted On'], errors='coerce')

# Тепер можна витягувати рік і місяць
df_outliers['Posted_Year'] = df_outliers['Posted On'].dt.year
df_outliers['Posted_Month'] = df_outliers['Posted On'].dt.month

In [78]:
# Обчислюємо кореляції з цільовою змінною (standrad)
corr_matrix_std = df_standard.corr(numeric_only=True)
top_corr_std = corr_matrix_std['Rent'].abs().sort_values(ascending=False)

# Відкидаємо саму Rent
top_corr_std = top_corr_std.drop('Rent')

# Вибираємо ознаки з кореляцією > 0.01
top_corr_features_std = top_corr_std[top_corr_std > 0.01].index.tolist()

In [79]:
# Обчислюємо кореляції з цільовою змінною (outliers)
corr_matrix_out = df_outliers.corr(numeric_only=True)
top_corr_out = corr_matrix_out['Rent'].abs().sort_values(ascending=False)

# Відкидаємо саму Rent
top_corr_out = top_corr_out.drop('Rent')

# Вибираємо ознаки з кореляцією > 0.01
top_corr_features_out = top_corr_out[top_corr_out > 0.01].index.tolist()

In [81]:
features_std = top_corr_features_std
# Визначаємо X (ознаки) та y (ціль)
X = df_standard[features_std]  # Ознаки
y = df_standard['Rent']  # Цільова змінна

print(f"\nРозмір X (ознак): {X.shape}")
print(f"Розмір y (цілі): {y.shape}")


Розмір X (ознак): (4226, 17)
Розмір y (цілі): (4226,)


In [85]:
features_std

['Total Floors',
 'City_Mumbai',
 'Bathroom',
 'Floor Level',
 'BHK',
 'Size',
 'City_Kolkata',
 'Furnishing_Status_Ordinal',
 'Tenant Preferred_Bachelors/Family',
 'Posted_Month',
 'Furnishing Status_Unfurnished',
 'Tenant Preferred_Bachelors',
 'Tenant Preferred_Family',
 'City_Chennai',
 'City_Bangalore',
 'Furnishing Status_Semi-Furnished',
 'City_Hyderabad']

In [82]:
features_out = top_corr_features_out
# Визначаємо X (ознаки) та y (ціль)
X = df_outliers[features_out]  # Ознаки
y = df_outliers['Rent']  # Цільова змінна

print(f"\nРозмір X (ознак): {X.shape}")
print(f"Розмір y (цілі): {y.shape}")


Розмір X (ознак): (520, 16)
Розмір y (цілі): (520,)


In [86]:
features_out

['Size',
 'BHK',
 'Bathroom',
 'City_Bangalore',
 'Furnishing Status_Semi-Furnished',
 'Furnishing Status_Unfurnished',
 'Total Floors',
 'Floor Level',
 'City_Chennai',
 'City_Delhi',
 'Tenant Preferred_Bachelors',
 'Tenant Preferred_Bachelors/Family',
 'Furnishing_Status_Ordinal',
 'City_Hyderabad',
 'Posted_Month',
 'City_Mumbai']

In [87]:
# Створюємо та застосовуємо скейлер
scaler_std = StandardScaler()
X_scaled_std = scaler.fit_transform(X)

# Перетворюємо назад у DataFrame для зручності
X_scaled_df_std = pd.DataFrame(X_scaled_std, columns=X.columns, index=X.index)

In [88]:
# Створюємо та застосовуємо скейлер
scaler_out = StandardScaler()
X_scaled_out = scaler.fit_transform(X)

# Перетворюємо назад у DataFrame для зручності
X_scaled_df_out = pd.DataFrame(X_scaled_out, columns=X.columns, index=X.index)

In [100]:
# Розділяємо дані: 80% на навчання, 20% на тест
X_train_std, X_test_std, y_train_std, y_test_std = train_test_split(
    X_scaled_df_std, y,
    test_size=0.2,  # 20% даних йде на тест
    random_state=42  # фіксуємо випадковість для відтворюваності
)

In [90]:
# Розділяємо дані: 80% на навчання, 20% на тест
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(
    X_scaled_df_out, y,
    test_size=0.2,  # 20% даних йде на тест
    random_state=42  # фіксуємо випадковість для відтворюваності
)

In [97]:
from sklearn.linear_model import LinearRegression

# Створюємо модель
model_std = LinearRegression()

# Навчаємо модель на навчальних даних
model_std.fit(X_train_std, y_train_std)

LinearRegression()

In [101]:
# Прогнози на навчальній вибірці
y_train_pred_std = model_std.predict(X_train_std)

# Прогнози на тестовій вибірці (нові дані!)
y_test_pred_std = model_std.predict(X_test_std)

# Порівняння перших 10 прогнозів з реальністю
comparison_std = pd.DataFrame({
    'Реальна рента': y_test_std.values[:20],
    'Прогнозована рента': y_test_pred_std[:20].round(0),
    'Помилка': (y_test_std.values[:20] - y_test_pred_std[:20]).round(0)
})
print("Приклади прогнозів на тестовій вибірці:")
print(comparison_std)

Приклади прогнозів на тестовій вибірці:
    Реальна рента  Прогнозована рента   Помилка
0           72000             94944.0  -22944.0
1           85000            141421.0  -56421.0
2           85000             92706.0   -7706.0
3          150000            156256.0   -6256.0
4          230000            310805.0  -80805.0
5          120000            141175.0  -21175.0
6          680000            223508.0  456492.0
7           95000             99460.0   -4460.0
8           70000            139784.0  -69784.0
9          100000            357325.0 -257325.0
10         180000            141757.0   38243.0
11         500000            355478.0  144522.0
12          80000            178730.0  -98730.0
13          85000             85548.0    -548.0
14         100000            143126.0  -43126.0
15          75000            217851.0 -142851.0
16         180000            259142.0  -79142.0
17         170000            175775.0   -5775.0
18          70000            122661.0  -52661.0


In [105]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Розраховуємо метрики для тестової вибірки
mae_std = mean_absolute_error(y_test_std, y_test_pred_std)
mse_std = mean_squared_error(y_test_std, y_test_pred_std)
rmse_std = np.sqrt(mse_std)
r2_std = r2_score(y_test_std, y_test_pred_std)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):')
print('='*50)
print(f'\nMAE: {mae_std:.2f} рента')
print(f'RMSE: {rmse_std:.2f} рента')
print(f'R²: {r2_std:.3f}')

# Порівняння з навчальною вибіркою
mae_std = mean_absolute_error(y_train_std, y_train_pred_std)
mse_std = mean_squared_error(y_train_std, y_train_pred_std)
rmse_std = np.sqrt(mse_std)
r2_train_std = r2_score(y_train_std, y_train_pred_std)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:')
print('='*50)
print(f'\nMAE: {mae_std:.2f} рента')
print(f'RMSE: {rmse_std:.2f} рента')
print(f'R²: {r2_train_std:.3f}')

МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):

MAE: 70075.92 рента
RMSE: 125165.90 рента
R²: 0.380
МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:

MAE: 66560.99 рента
RMSE: 184147.15 рента
R²: 0.117


In [106]:
from sklearn.linear_model import LinearRegression

# Створюємо модель
model_out = LinearRegression()

# Навчаємо модель на навчальних даних
model_out.fit(X_train_out, y_train_out)

LinearRegression()

In [110]:
# Прогнози на навчальній вибірці
y_train_pred_out = model_out.predict(X_train_out)

# Прогнози на тестовій вибірці (нові дані!)
y_test_pred_out = model_out.predict(X_test_out)

# Порівняння перших 10 прогнозів з реальністю
comparison_out = pd.DataFrame({
    'Реальна рента': y_test_out.values[:20],
    'Прогнозована рента': y_test_pred_out[:20].round(0),
    'Помилка': (y_test_out.values[:20] - y_test_pred_out[:20]).round(0)
})
print("Приклади прогнозів на тестовій вибірці:")
print(comparison_out)

Приклади прогнозів на тестовій вибірці:
    Реальна рента  Прогнозована рента   Помилка
0           72000             94944.0  -22944.0
1           85000            141421.0  -56421.0
2           85000             92706.0   -7706.0
3          150000            156256.0   -6256.0
4          230000            310805.0  -80805.0
5          120000            141175.0  -21175.0
6          680000            223508.0  456492.0
7           95000             99460.0   -4460.0
8           70000            139784.0  -69784.0
9          100000            357325.0 -257325.0
10         180000            141757.0   38243.0
11         500000            355478.0  144522.0
12          80000            178730.0  -98730.0
13          85000             85548.0    -548.0
14         100000            143126.0  -43126.0
15          75000            217851.0 -142851.0
16         180000            259142.0  -79142.0
17         170000            175775.0   -5775.0
18          70000            122661.0  -52661.0


In [111]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Розраховуємо метрики для тестової вибірки
mae_out = mean_absolute_error(y_test_out, y_test_pred_out)
mse_out = mean_squared_error(y_test_out, y_test_pred_out)
rmse_out = np.sqrt(mse_out)
r2_out = r2_score(y_test_out, y_test_pred_out)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):')
print('='*50)
print(f'\nMAE: {mae_out:.2f} рента')
print(f'RMSE: {rmse_out:.2f} рента')
print(f'R²: {r2_out:.3f}')

# Порівняння з навчальною вибіркою
mae_out = mean_absolute_error(y_train_out, y_train_pred_out)
mse_out = mean_squared_error(y_train_out, y_train_pred_out)
rmse_out = np.sqrt(mse_out)
r2_train_out = r2_score(y_train_out, y_train_pred_out)

print('='*50)
print('МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:')
print('='*50)
print(f'\nMAE: {mae_out:.2f} рента')
print(f'RMSE: {rmse_out:.2f} рента')
print(f'R²: {r2_train_out:.3f}')

МЕТРИКИ ЯКОСТІ МОДЕЛІ (на тестовій вибірці):

MAE: 70075.92 рента
RMSE: 125165.90 рента
R²: 0.380
МЕТРИКИ ЯКОСТІ МОДЕЛІ на тренувальній вибірці:

MAE: 66560.99 рента
RMSE: 184147.15 рента
R²: 0.117


####Як можна покращити модель
- перша ідея була - розділити вибірки на стандартну та викиди і оцінити кожну окремо (перевірила - невдала ідея)
- можна логарифмувати ренту, бо маємо високі значення. це має стабілізувати значення і модель має працювати адекватніше
- зробити додаткову колонку з ціною за квадратний фунт